In [1]:
'''
Created on 2018. 2. 14.

@author: phs
'''
"""
1.대화 말뭉치 파일을 읽어들인다.
2.대화 말뭉치를 읽어서 자연어 처리 및  Bag of word 생성
3.Bag of word를 딥러닝 알고리즘 활용을 위한 입력으로 변환
4.딥러닝(tensorflow)을 통한 자연어 이해 모델 생성
5.자연어 이해 모델을 관리한다(저장,읽기)
"""
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# things we need for NLP
import json
import pickle
import random

# import nltk
# from nltk.stem.lancaster import LancasterStemmer
# stemmer = LancasterStemmer()

# from konlpy.tag import Komoran
from konlpy.tag import Okt
# komoran = Komoran()
twitter = Okt()

import tflearn
import tensorflow as tf
import numpy as np



C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\tensorf

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
def read_dialog_intents_jsonfile(input_file_name):
    """
     대화 말뭉치 파일을 읽어들인다.
    """
    with open(input_file_name, 'rt', encoding='UTF8') as json_data:
        intents = json.load(json_data)
        
    return intents
    


In [3]:
def dialog_nlp_processing(intents):
    """
     대화 말뭉치를 읽어서 자연어 처리 및  Bag of word 생성
    """
    words = []
    classes = []
    documents = []
    ignore_words = ['?']
    # loop through each sentence in our intents patterns
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            # tokenize each word in the sentence
#             w = nltk.word_tokenize(pattern)
            pos_result = twitter.pos(pattern, norm=True, stem=True)
            w = [lex for lex, pos in pos_result]
            # add to our words list
            words.extend(w)
            # add to documents in our corpus
            documents.append((w, intent['tag']))
            # add to our classes list
            if intent['tag'] not in classes:
                classes.append(intent['tag'])
    
    # stem and lower each word and remove duplicates
#     words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
#     words = sorted(list(set(words)))
    words = [w for w in words if w not in ignore_words]
    words = sorted(list(set(words)))
    
    # remove duplicates
    classes = sorted(list(set(classes)))
    
    return classes, documents, words



In [4]:
def prepare_machine_learning(classes, documents, words):
    """
    Bag of word를 딥러닝 알고리즘 활용을 위한 입력으로 변환
    """
    
    # create our training data
    training = []
    output_row = []
    # create an empty array for our output
    output_empty = [0] * len(classes)
    
    # training set, bag of words for each sentence
    for doc in documents:
        # initialize our bag of words
        bag = []
        # list of tokenized words for the pattern
        pattern_words = doc[0]
        # stem each word
#         pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
        # create our bag of words array
        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)
    
        # output is a '0' for each tag and '1' for current tag
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
    
        training.append([bag, output_row])
    
    # shuffle our features and turn into np.array
    random.shuffle(training)
    training = np.array(training)
    
    # create train and test lists
    train_x = list(training[:,0])
    train_y = list(training[:,1])

    return train_x, train_y



In [5]:
def create_tensorflow_learning_model(train_x, train_y, output_model_file_name):
    """
    딥러닝(tensorflow)을 통한 자연어 이해 모델 생성
    """
    
    # reset underlying graph data
    #tf.compat.v1.reset_default_graph()
    tf.reset_default_graph()
    # Build neural network
    net = tflearn.input_data(shape=[None, len(train_x[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
    net = tflearn.regression(net)
    
    # Define model and setup tensorboard
    model = tflearn.DNN(net, tensorboard_dir='home_tflearn_kr_logs')
    # Start training (apply gradient descent algorithm)
    model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

#     # Using Keras, Build neural network
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(8, input_shape=(len(train_x[0]),)),
#         tf.keras.layers.Dense(8),
#         tf.keras.layers.Dense(len(train_y[0]), activation="softmax"),
#         ])
    
#     model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
#     model.fit(train_x, train_y, epochs=1000, batch_size=8)    

    # save the trained model to directory
    model.save(output_model_file_name)

    return model



In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
#     sentence_words = nltk.word_tokenize(sentence)
    pos_result = twitter.pos(sentence, norm=True, stem=True)
    sentence_words = [lex for lex, pos in pos_result]
    # stem each word
#     sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words



In [7]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [8]:
# save all of our data structures
def save_training_data_structures(words, classes, train_x, train_y, output_training_data_file_name):
    """
    자연어 이해 모델을 관리한다(저장,읽기)
    """
    # save all of our data structures
    pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( output_training_data_file_name, "wb" ) )



In [10]:
# 대화 말뭉치 파일을 읽어들인다.
input_file_name = './DialogIntents/intents_home_kr.json'
intents = read_dialog_intents_jsonfile(input_file_name)
    
# 대화 말뭉치를 읽어서 자연어 처리 및  Bag of word 생성
classes, documents, words = dialog_nlp_processing(intents)
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
    


54 documents
11 classes ['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
77 unique stemmed words ['.', '가', '감사', '감사하다', '개발', '개선', '거기', '경력', '경험', '계세', '계시다', '고맙다', '과', '그렇다', '꺼지다', '나중', '날', '누구', '는', '다음', '닥치다', '돈', '들리다', '또', '를', '많이', '멍청이', '무슨', '바보', '버세', '번', '보완', '부분', '부족', '부탁', '사라지다', '사항', '새끼', '서비스', '소개', '수고', '안녕', '안녕하다', '야', '약점', '어떻다', '어학', '언제', '업적', '에', '에요', '여기', '영어', '요', '은', '을', '이', '이다', '일본어', '임마', '있다', '자기', '자다', '자신', '장사', '장점', '전공', '제품', '졸업', '좋다', '줄다', '중국어', '지내다', '친절하다', '하다', '학교', '해주다']


In [11]:
# Bag of word를 딥러닝 알고리즘 활용을 위한 입력으로 변환
train_x, train_y = prepare_machine_learning(classes, documents, words)

C:\Users\saint\Anaconda3\envs\chat_env\lib\site-packages\ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [12]:
print(train_x[0])
print(len(train_x[0]))
print(train_y[0])
print(len(train_y[0]))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
77
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
11


In [13]:
# 딥러닝(tensorflow)을 통한 자연어 이해 모델 생성
output_model_file_name = './NLUModel/model_home_kr.tflearn'
model = create_tensorflow_learning_model(train_x, train_y, output_model_file_name)


Training Step: 6999  | total loss: 0.10495 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.10495 - acc: 0.9561 -- iter: 48/54
Training Step: 7000  | total loss: 0.09561 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.09561 - acc: 0.9605 -- iter: 54/54
--
INFO:tensorflow:C:\Users\saint\GroupProject\step5\jupyter_app\nlp\home_chat\NLUModel\model_home_kr.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [14]:
# 자연어 이해 모델을 관리한다(저장,읽기)
output_training_data_file_name = "./NLUModel/training_data_home_kr"
save_training_data_structures(words, classes, train_x, train_y, output_training_data_file_name)


In [15]:
p = bow("비젼은 무엇입니까?", words)
print("p is Bag of word for '비젼은 무엇입니까?' :{}".format(p))
print("classes :{}".format(classes))
    
print("model.predict([p]) :{}".format(model.predict([p])))
    
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

p is Bag of word for '비젼은 무엇입니까?' :[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
classes :['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
model.predict([p]) :[[4.2046016e-11 4.0230997e-08 1.4021193e-12 9.6152912e-11 2.9819619e-04
  2.9969469e-01 6.5913968e-02 1.6836681e-05 4.1041709e-03 6.2702901e-07
  6.2997150e-01]]
54 documents
11 classes ['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
77 unique stemmed words ['.', '가', '감사', '감사하다', '개발', '개선', '거기', '경력', '경험', '계세', '계시다', '고맙다', '과', '그렇다', '꺼지다', '나중', '날', '누구', '는', '다음', '닥치다', '돈', '들리다', '또', '를', '많이', '멍청이', '무슨', '바보', '버세', '번', '보완', '부분', '부족', '부탁', '사라지다', '사항', '새끼', '서비스', '소개', '수고', '안녕', '안녕하다', '야', '약점', '어떻다', '어학', '언제', '업적', '에', '에요', '여기', '영어', '요', '은', '을', '이', '이다', '일본어', '임마', '있다', '자기', '자다', '자신', '장사', '장